In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
from matplotlib import pyplot as plt
import folium

In [9]:
anxiety = pd.read_csv("Data/anxiety-disorders.csv")
anxiety

,Entity,Code,Year,Anxiety disorders (share of population) - Sex: Both - Age: Age-standardized,"GDP per capita, PPP (constant 2017 international $)",Population (historical estimates),Continent
0,Abkhazia,OWID_ABK,2015,NaN,NaN,NaN,Asia
1,Afghanistan,AFG,1990,4.713314,NaN,10694804.0,NaN
2,Afghanistan,AFG,1991,4.702100,NaN,10745168.0,NaN
3,Afghanistan,AFG,1992,4.683743,NaN,12057436.0,NaN
4,Afghanistan,AFG,1993,4.673549,NaN,14003764.0,NaN
...,...,...,...,...,...,...,...
58665,Zimbabwe,ZWE,1986,NaN,NaN,8983048.0,NaN
58666,Zimbabwe,ZWE,1987,NaN,NaN,9277484.0,NaN
58667,Zimbabwe,ZWE,1988,NaN,NaN,9568745.0,NaN
58668,Zimbabwe,ZWE,1989,NaN,NaN,9846352.0,NaN


In [10]:
anxiety = anxiety.dropna(
    subset=[
        "Anxiety disorders (share of population) - Sex: Both - Age: Age-standardized",
        "GDP per capita, PPP (constant 2017 international $)",
    ]
)

In [11]:
anxiety = pd.merge(anxiety, countries, left_on="Code", right_on="iso_a3")
anxiety = GeoDataFrame(anxiety, geometry=anxiety["geometry"])
anxiety = anxiety.copy()
anxiety["geometry"] = anxiety["geometry"].centroid
anxiety = anxiety.sort_values("Year", ascending=True)
longitude, latitude = anxiety["geometry"].x, anxiety["geometry"].y
anxiety["longitude"] = longitude
anxiety["latitude"] = latitude

/var/folders/ny/x98qxh1124dbtz5g3f6s09hh0000gn/T/ipykernel_84631/3761927194.py:4: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




In [12]:
scatter_anxiety = px.scatter(
    anxiety.dropna(
        subset=[
            "Anxiety disorders (share of population) - Sex: Both - Age: Age-standardized",
            "GDP per capita, PPP (constant 2017 international $)",
        ]
    ),
    color="continent",
    size="pop_est",
    color_continuous_scale=px.colors.cyclical.IceFire,
    animation_frame="Year",
    animation_group="Code",
    hover_name="Entity",
    size_max=35,
    x="GDP per capita, PPP (constant 2017 international $)",
    y="Anxiety disorders (share of population) - Sex: Both - Age: Age-standardized",
    log_x=True,
)
scatter_anxiety.update_layout(yaxis_title="Anxiety disorders (share of population)")
scatter_anxiety.update_xaxes(showgrid=False)
scatter_anxiety.update_yaxes(showgrid=False)
scatter_anxiety.show()